In [ ]:
Analysis of PONG reinforcement learning algorithm


The idea is to find a hyperparameters that train the model to be competitive with the AI as quickly as possiable.
The hope is to be able to find a model that can be used as part of a workshop.  Ideally I'd like the training to take only as long as the workshop, so we can start it at the beginning, and check it at the end.  (which would be about 2 hours)

I define 'competitive with the AI' as obtaining a mean rewards >= 0.0 over 100 episodes.

Depending on the analysis, it might be possiable to train the model faster using parallism.  I'd be happy to rent up to 32 cores, so we should be able to get enough CPU time.  Might have to shift to TF for this, and might have to parallise the enviroment evaluation (this could be done by launching multiple threads and running each episode in the batch independantly... I kind of like the idea of this).